In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
data=pd.read_csv("/content/Customer Conversion Prediction - Customer Conversion Prediction.csv")
data.shape

(45211, 11)

In [4]:
data.head()

,age,job,marital,education_qual,call_type,day,mon,dur,num_calls,prev_outcome,y
0,58,management,married,tertiary,unknown,5,may,261,1,unknown,no
1,44,technician,single,secondary,unknown,5,may,151,1,unknown,no
2,33,entrepreneur,married,secondary,unknown,5,may,76,1,unknown,no
3,47,blue-collar,married,unknown,unknown,5,may,92,1,unknown,no
4,33,unknown,single,unknown,unknown,5,may,198,1,unknown,no


In [5]:
data["mon"].unique()

array(['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'jan', 'feb',
       'mar', 'apr', 'sep'], dtype=object)

In [6]:
data.isnull().sum()

age               0
job               0
marital           0
education_qual    0
call_type         0
day               0
mon               0
dur               0
num_calls         0
prev_outcome      0
y                 0
dtype: int64

In [7]:
data.dtypes

age                int64
job               object
marital           object
education_qual    object
call_type         object
day                int64
mon               object
dur                int64
num_calls          int64
prev_outcome      object
y                 object
dtype: object

In [8]:
data=data.drop_duplicates()

In [9]:
data

,age,job,marital,education_qual,call_type,day,mon,dur,num_calls,prev_outcome,y
0,58,management,married,tertiary,unknown,5,may,261,1,unknown,no
1,44,technician,single,secondary,unknown,5,may,151,1,unknown,no
2,33,entrepreneur,married,secondary,unknown,5,may,76,1,unknown,no
3,47,blue-collar,married,unknown,unknown,5,may,92,1,unknown,no
4,33,unknown,single,unknown,unknown,5,may,198,1,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,cellular,17,nov,977,3,unknown,yes
45207,71,retired,divorced,primary,cellular,17,nov,456,2,unknown,yes
45208,72,retired,married,secondary,cellular,17,nov,1127,5,success,yes
45209,57,blue-collar,married,secondary,telephone,17,nov,508,4,unknown,no


In [10]:
from sklearn.preprocessing import LabelEncoder
del data["job"]
dft=data
dft["marital"]=dft["marital"].map({"married" : 0 , "single" :1 , "divorced" : 2})
dft["education_qual"]=dft["education_qual"].map({'tertiary' : 3 , 'secondary' : 2 , 'unknown' : 0 , 'primary' : 1  })
dft["call_type"]=dft["call_type"].map({'unknown' : 0, 'cellular' : 1, 'telephone' : 2})
dft["prev_outcome"]=dft["prev_outcome"].map({'unknown' : 2 , 'failure' :0 , 'other' : 3 , 'success' : 1})
dft["mon"]=dft["mon"].map({'jan' : 1 , 'feb' : 2 ,'mar' : 3 ,'apr' : 4 ,'may' : 5 , 'jun' : 6 , 'jul' : 7 , 'aug' : 8 , 'sep' : 9 , 'oct' : 10 , 'nov' : 11 , 'dec' : 12 ,})

dft["y"]=dft["y"].map({'no' : 0, 'yes' :1 })



In [11]:
dft

,age,marital,education_qual,call_type,day,mon,dur,num_calls,prev_outcome,y
0,58,0,3,0,5,5,261,1,2,0
1,44,1,2,0,5,5,151,1,2,0
2,33,0,2,0,5,5,76,1,2,0
3,47,0,0,0,5,5,92,1,2,0
4,33,1,0,0,5,5,198,1,2,0
...,...,...,...,...,...,...,...,...,...,...
45206,51,0,3,1,17,11,977,3,2,1
45207,71,2,1,1,17,11,456,2,2,1
45208,72,0,2,1,17,11,1127,5,1,1
45209,57,0,2,2,17,11,508,4,2,0


In [12]:
 from sklearn.model_selection import train_test_split
 x_train, x_test, y_train, y_test=train_test_split(dft[dft.columns[:-1]].values,dft['y'].values,test_size=0.25)

In [13]:
x_train.shape, x_test.shape

((33903, 9), (11302, 9))

In [14]:
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
x_train= std.fit_transform( x_train)
x_test = std.transform(x_test)

In [24]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
model1 = LogisticRegression(random_state=1)
model2 = tree.DecisionTreeClassifier(random_state=1)
model3 = KNeighborsClassifier(3)
model =  VotingClassifier(estimators=[("lr",model1),("dt", model2), ("knn", model3)],voting="soft")
model.fit(x_train, y_train)
preds = model.predict(x_test)


In [26]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,model.predict_proba(x_test)[:,1])

0.8935924466439026